In [1]:
import torch

In [2]:
import math
from typing import List, Optional, Union

## 1 num_swapped_pairs 

функция для расчёта количества неправильно упорядоченных пар (корректное упорядочивание - от наибольшего значения в ys_true к меньшему), или переставленных пар . Не забудьте, что одну и ту же пару не нужно учитывать дважды. 

In [3]:
ys_true = torch.rand(5)
ys_pred = torch.rand(5)

In [4]:
def num_swapped_pairs (ys_true: torch.Tensor, ys_pred: torch.Tensor):
    #сортируем по предсказанию
    ys_pred = ys_pred.sort(descending=True)
    #берем индексы из предсказания и получаем фактический порядок
    ys_true = ys_true[ys_pred.indices]
    count = 0
    #считаем сколько значений ys_true стоят неправильно (меньше )
    for i in range(len(ys_true )):
        for j in range(i, len(ys_true )):
            if ys_true [i] < ys_true [j]:
                count += 1
    return count

In [6]:
num_swapped_pairs_m(ys_true, ys_pred)

2

## 2 compute_gain 

compute_gain - вспомогательная функция для расчёта DCG  и NDCG, рассчитывающая показатель gain. Принимает на вход дополнительный аргумент - указание схемы начисления gain (gain_scheme).

В лекции был приведен пример константного начисления, равного в точности оценке релевантности. Необходимо реализовать как этот метод (при gain_scheme="const") начисления gain, так и экспоненциальный (gain_scheme="exp2"), рассчитываемый по формуле (2 
r−1), где r - реальная релевантность документа некоторому запросу. Логика здесь такова, что чем выше релевантность, тем ценнее объект, и темп роста “ценности" нелинеен - гораздо важнее отделить документ с релевантностью 5 от документа с релевантностью 3, нежели 3 от 1 (ведь они оба мало релевантны).

Функция принимает на вход единственное число (не тензор).


dcg  и ndcg -  функции расчета DCG и NDCG соответственно. Принимают на вход дополнительный параметр gain_scheme, аналогичный таковому в функции compute_gain (её необходимо использовать при расчетах этих метрик). Для NDCG разрешается переиспользовать функцию расчёта DCG. Для совпадения ответа с системой оценки рекомендуется использовать логарифм из модуля math.

In [7]:
def compute_gain (y_value: float, gain_scheme="const"):
    if gain_scheme=="const":
        d = y_value
    elif gain_scheme=="exp2":
        d = 2**y_value - 1
    return d

In [8]:
compute_gain( 0.3, gain_scheme="const")

0.3

In [9]:
compute_gain ( 0.3, gain_scheme="exp2")

0.2311444133449163

In [10]:
compute_gain ( 0.3, "exp2")

0.2311444133449163

## 3 dcg

dcg  и ndcg -  функции расчета DCG и NDCG соответственно. Принимают на вход дополнительный параметр gain_scheme, аналогичный таковому в функции compute_gain (её необходимо использовать при расчетах этих метрик).


Для NDCG разрешается переиспользовать функцию расчёта DCG. Для совпадения ответа с системой оценки рекомендуется использовать логарифм из модуля math.

In [11]:
def dcg (ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str):
    ## сортировка y_true по y_pred
    ys_true = ys_true[ys_pred.sort(descending=True).indices].double()
    ## значение предикт остортированные по y_true 
    if gain_scheme == "const":
        gain = torch.Tensor([compute_gain(x, "const") for x in ys_true]).double()
    elif gain_scheme == "exp2":
        gain = torch.Tensor([compute_gain(x, "exp2") for x in ys_true]).double()
    discount = torch.Tensor ([math.log(x, 2) for x in torch.arange(len(ys_true))+2]).double()
    res = sum(gain/discount)
    return  res

In [12]:
def ndcg (ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str):
    dcg_1 = dcg (ys_true, ys_pred, gain_scheme)
    ## идеальное расположение текущих 
    ys_true = ys_true.sort(descending=True).values.double()
    ## значение предикт остортированные по y_true 
    if gain_scheme == "const":
        gain = torch.Tensor([compute_gain(x, "const") for x in ys_true]).double()
    elif gain_scheme == "exp2":
        gain = torch.Tensor([compute_gain(x, "exp2") for x in ys_true]).double()
        
    discount = torch.Tensor ([math.log(x, 2) for x in torch.arange(len(ys_true))+2]).double()
    res = sum(gain/discount)
    return  dcg_1 / res

## precission_at_k 

функция расчёта точности в топ-k позиций для бинарной разметки (в ys_true содержатся только нули и единицы). Если среди лейблов нет ни одного релевантного документа (единицы), то нужно вернуть -1. 

Функция принимает на вход параметр k, указывающий на то, по какому количеству объектов необходимо произвести расчёт метрики. Учтите, что k может быть больше количества элементов во входных тензорах. 

При реализации precission_at_k необходимо добиться того, что максимум функции в единице был достижим при любом ys_true, за исключением не содержащего единиц. В силу этой особенности не рекомендуется переиспользовать имплементацию precission_at_k в других метриках настоящего ДЗ.

In [13]:
ys_true = torch.Tensor([1, 0, 1, 1, 0, 0, 0, 0, 0])
ys_true

tensor([1., 0., 1., 1., 0., 0., 0., 0., 0.])

In [14]:
ys_pred = torch.rand(9)
ys_pred

tensor([0.1125, 0.6432, 0.5977, 0.0965, 0.9104, 0.0099, 0.8463, 0.6246, 0.8025])

In [15]:
def precission_at_k(ys_true: torch.Tensor, ys_pred: torch.Tensor, k: int) -> float:
    #если нет истинных релевантных объектов (1)
    if ys_true.sum() == 0:
        return -1
    #сортируем все наши предсказания и берем их индексы 
    _, argsort = torch.sort(ys_pred, descending=True, dim=0)
    #сортируем ys_true по индексу ys_pred
    ys_true_sorted = ys_true[argsort]
    #сколько у нас количество хитов (1) в топ k из истинных объектов
    hits = ys_true_sorted[:k].sum()
    prec = hits / min(ys_true.sum(), k) #делим на ys_true.sum() если ys_true.sum() < k
    #сценарий 1: мы угадали не все единицы, тогда делится на общее количество k (стандартный вариант) - наш порог 
    #сценарий 2: мы угадали все единицы и дальшее идут одни нули в топ k, то есть ys_true.sum() < k и тогда precision = 1 
    return float(prec)

In [23]:
_, argsort = torch.sort(ys_pred, descending=True, dim=0)

In [24]:
argsort

tensor([4, 6, 8, 1, 7, 2, 0, 3, 5])

In [25]:
ys_true[argsort]

tensor([0., 0., 0., 0., 0., 1., 1., 1., 0.])

## reciprocal_rank 

функция для расчёта MRR (без усреднения, т.е. для одного запроса и множества документов). В ys_true могут содержаться только нули и максимум одна единица. 

In [63]:
ys_true = torch.Tensor([0, 0, 0, 0, 0, 0, 1, 0, 0])
ys_true

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [64]:
ys_pred = torch.rand(9)
ys_pred

tensor([0.1830, 0.9609, 0.0061, 0.2294, 0.5985, 0.7781, 0.1065, 0.5801, 0.6598])

In [69]:
def reciprocal_rank(ys_true, ys_pred) -> float:
    if ys_true.sum() == 0:
        return 0
    _, ys_pred_index = torch.sort(ys_pred, descending=True)
    ys_true_sorted = ys_true[ys_pred_index]
    position = 0
    for i in range(len(ys_true_sorted)):
        if ys_true_sorted[i] == 1:
            position = i + 1
    return 1/position

## p_found

p_found - функция расчета P-found от Яндекса, принимающая на вход дополнительный параметр p_break - вероятность прекращения просмотра списка документов в выдаче. Базовая вероятность просмотреть первый документ в выдаче (pLook[0] в лекции) равняется единице. ys_true нормированы от 0 до 1 (вероятность удовлетворения запроса пользователя). 

In [72]:
ys_true = torch.Tensor([1, 0, 1, 1, 0])

In [73]:
ys_pred = torch.Tensor([0.3, 0.2, 0.5, 0.1, 0.8])

In [86]:
def p_found(ys_true: torch.Tensor, ys_pred: torch.Tensor, p_break: float = 0.15 ) -> float:
    p_look = 1
    p_found = 0
    _, argsort = torch.sort(ys_pred, descending=True, dim=0)
    ys_true_sorted = ys_true[argsort]

    for cur_y in ys_true_sorted:
        p_found += p_look * float(cur_y)
        p_look = p_look * (1 - float(cur_y)) * (1 - p_break)
        print (p_look)
    
    return p_found

In [87]:
p_found(ys_true, ys_pred, 0.15 ) 

0.85
0.0
0.0
0.0
0.0


0.85

## average_precision 

Average Precision (AP) показывает, насколько много релевантных объектов сконцентрировано среди самых высокооцененных. Чувствительна к порядку ранжирования в топе.

average_precision - функция расчета AP для бинарной разметки (в ys_true содержатся только нули и единицы). Если среди лейблов нет ни одного релевантного документа (единицы), то нужно вернуть -1.

In [88]:
def average_precision(ys_true: torch.Tensor, ys_pred: torch.Tensor) -> float:
    if ys_true.sum() == 0:
        return -1
    _, argsort = torch.sort(ys_pred, descending=True, dim=0)
    ys_true_sorted = ys_true[argsort]
    rolling_sum = 0
    num_correct_ans = 0
    
    for idx, cur_y in enumerate(ys_true_sorted, start=1):
        if cur_y == 1:
            num_correct_ans += 1
            rolling_sum += num_correct_ans / idx
    if num_correct_ans == 0:
        return 0
    else:
        return rolling_sum / num_correct_ans

In [89]:
average_precision(ys_true, ys_pred)

0.5888888888888889